In [39]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
#import psycopg2
#from config import db_password
import time
import pandas as pd
import matplotlib as plt
from sklearn.datasets import make_blobs
import sklearn as skl
import tensorflow as tf

In [40]:
dpo_table = pd.read_csv('dpo_table.csv')
#pkg_dim_table = pd.read_csv('pkg_dims_table.csv')
#region = pd.read_csv('region_table.csv')

In [41]:
for column in dpo_table.columns:
    print(f"Column {column} has {dpo_table[column].isnull().sum()} null values")

Column actdely_dt has 0 null values
Column omstracknum has 0 null values
Column ordnum has 0 null values
Column origin_zip has 0 null values
Column dest_zip has 0 null values
Column shipnode has 0 null values
Column channel has 0 null values
Column carrier_scac has 0 null values
Column reqship_dt has 0 null values
Column sysship_dttm has 0 null values
Column sysship_dt has 0 null values
Column estdely_dt has 0 null values
Column actdely_month has 0 null values
Column estdely_week has 0 null values
Column delivered_late has 0 null values
Column xcpnscan1_desc has 0 null values
Column xcpnscan2_desc has 0 null values
Column xcpnscan3_desc has 0 null values
Column xcpnscan4_desc has 0 null values
Column planned_transit has 0 null values
Column carrier_pub_tnt has 0 null values
Column transit_override has 0 null values
Column carrier_xcp has 0 null values
Column guest_xcp has 0 null values
Column holiday_xcp has 0 null values
Column weather_xcp has 0 null values
Column uncontrol_xcp has 0 

In [47]:
dpo_table.head()

,actdely_dt,omstracknum,ordnum,origin_zip,dest_zip,shipnode,channel,carrier_scac,reqship_dt,sysship_dttm,...,ots_rsd,post_release_inf,node_mnr,carrier_not_meet_tnt,origin_defect,uncontrollable_defect,carrier_defect.1,origin_defect.1,system_defect.1,dpo_delivered_late
0,2019-02-03,1Z2E24F1YW37836926,1022331943153,17408,60126,9475,FC 9475,UPSN,2019-01-30,2019-01-29 20:25,...,0.0,0.0,Node: Manifested - not received,0,origin,0,0,origin,0,N
1,2019-02-03,420100169261290982827026490071,9250073345592,7047,10016,STR1865,STR,USPS,2019-02-01,2019-02-01 15:51,...,0.0,0.0,0,0,origin,0,0,origin,0,N
2,2019-02-03,420100259261290982827019489617,3550521247344,7047,10025,STR1865,STR,USPS,2019-02-02,2019-02-02 12:08,...,1.0,1.0,0,0,0,0,0,0,0,N
3,2019-02-03,420100269261290982827013614626,9050522892943,7047,10026,STR1865,STR,USPS,2019-02-02,2019-02-02 13:21,...,0.0,0.0,0,0,0,0,0,0,0,N
4,2019-02-03,420100289261290203606120962697,1050507158738,10451,10028,STR2475,STR,USPS,2019-02-01,2019-01-31 17:22,...,0.0,0.0,Node: Manifested - not received,0,origin,0,0,origin,0,N


In [46]:
dpo_table=dpo_table.fillna(0)

In [48]:
dpo_table.dtypes

actdely_dt                object
omstracknum               object
ordnum                     int64
origin_zip                 int64
dest_zip                   int64
shipnode                  object
channel                   object
carrier_scac              object
reqship_dt                object
sysship_dttm              object
sysship_dt                object
estdely_dt                object
actdely_month             object
estdely_week               int64
delivered_late            object
xcpnscan1_desc            object
xcpnscan2_desc            object
xcpnscan3_desc            object
xcpnscan4_desc            object
planned_transit          float64
carrier_pub_tnt          float64
transit_override         float64
carrier_xcp               object
guest_xcp                 object
holiday_xcp               object
weather_xcp               object
uncontrol_xcp             object
order_dttm                object
late_ship_business        object
node_on_time_ship         object
ots_rsd   

In [49]:
dpo_table = dpo_table.drop(['system_defect','carrier_defect','uncontrollable_defect.1', ], axis=1)

KeyError: "['system_defect' 'carrier_defect' 'uncontrollable_defect.1'] not found in axis"

In [50]:
# do NaN's mean the carrier did meet TNT? 
dpo_table.carrier_not_meet_tnt.value_counts()

0                           43
Carrier Did not meet TNT     7
Name: carrier_not_meet_tnt, dtype: int64

In [51]:
dpo_table.origin_defect.value_counts()

0         27
origin    23
Name: origin_defect, dtype: int64

In [52]:
dpo_table.dpo_delivered_late.value_counts()

N    36
Y    14
Name: dpo_delivered_late, dtype: int64

In [53]:

dpo_table['actdely_dt'] = pd.to_datetime(dpo_table['actdely_dt'])

In [54]:
y = dpo_table['dpo_delivered_late']
X = dpo_table.drop(columns = 'dpo_delivered_late')

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y, random_state=1, stratify=y)

In [56]:
X_train.shape

(37, 39)

In [57]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
   max_iter=200,
   random_state=1)

In [58]:
classifier.fit(X_train, y_train)

TypeError: float() argument must be a string or a number, not 'Timestamp'